# Deprecated due to poor performance

# Get all scenes with more people detected

In [3]:
import cv2
from IPython.display import display, Image
import glob

import numpy as np
import onnxruntime as ort
import glob
from ipywidgets import FloatProgress
from datetime import datetime

TIME_INTERVAL = 5.0 # second(s) for down-sampling
PEOPLE_THRESHOLD = 10 # number of people in a frame to be considered as a crowd

w = "YOLO/yolov7.onnx"
cuda = False

#YoloV7 attributes
videos = []
names = ['person']
colors = {'person' : [242, 82, 141]}

def processVideo(pathIn, pathOut, thumbPath=None):
	VIDEO_NAME = pathIn.split('/')[-1]
	if VIDEO_NAME.endswith('.mp4'):
		VIDEO_NAME = VIDEO_NAME[:-4]

	with open('./crowd/processed_videos.csv', 'r') as f_object:
		text = f_object.readline()
		while text != "":
				text = f_object.readline()
				if text == VIDEO_NAME:
					f_object.close()
					return #skip processed videos
		f_object.close()
	
	current_time = datetime.now().strftime("%H:%M:%S")
	print(f"Starting to process {VIDEO_NAME}, {current_time}")

	count = 0
	vidcap = cv2.VideoCapture(pathIn)
	success,image = vidcap.read()
	success = True
	total_secs = vidcap.get(cv2.CAP_PROP_FRAME_COUNT) / vidcap.get(cv2.CAP_PROP_FPS)

	p_bar = FloatProgress(min=0, max=total_secs)
	display(p_bar)

	while success:
		pos = count*TIME_INTERVAL # seconds
		if pos > total_secs:
			break
		
		vidcap.set(cv2.CAP_PROP_POS_MSEC, pos * 1000)
		p_bar.value = pos
		p_bar.description = f"{int(pos)}/{int(total_secs)}"
		success,image = vidcap.read()
		
		if success:
			person_in_frame, thumb = get_person_count_in_image(image, w, cuda=cuda)
			if person_in_frame > PEOPLE_THRESHOLD:
				cv2.imwrite(pathOut + f"/{VIDEO_NAME}-{count}_{person_in_frame}.jpg", image)
				print(f"Found a crowd with {person_in_frame} detectable persons in {VIDEO_NAME} at {count} seconds")
				with open('./crowd/found_frames.csv', 'a') as f_object:
					f_object.write("{},{},{}\n".format(VIDEO_NAME, count, person_in_frame))
					f_object.close()
				if thumb is not None and thumbPath is not None:
					cv2.imwrite(thumbPath + f"/{VIDEO_NAME}-t{count}_{person_in_frame}.jpg", thumb)
		
		count = count + 1
    
	p_bar.close()
	vidcap.release()
	with open('./crowd/processed_videos.csv', 'a') as f_object:
		f_object.write(VIDEO_NAME + '\n')
		f_object.close()

def letterbox(im, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleup=True, stride=32):
	# Resize and pad image while meeting stride-multiple constraints
	shape = im.shape[:2]  # current shape [height, width]
	if isinstance(new_shape, int):
		new_shape = (new_shape, new_shape)
	# Scale ratio (new / old)
	r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
	if not scaleup:  # only scale down, do not scale up (for better val mAP)
		r = min(r, 1.0)
	# Compute padding
	new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
	dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding

	if auto:  # minimum rectangle
		dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding

	dw /= 2  # divide padding into 2 sides
	dh /= 2

	if shape[::-1] != new_unpad:  # resize
		im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
	top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
	left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
	im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
	return im, r, (dw, dh)
	
def get_person_count_in_image(img, w, cuda=False):
	providers = ['CoreMLExecutionProvider', 'CPUExecutionProvider'] if cuda else ['CPUExecutionProvider']
	session = ort.InferenceSession(w, providers=providers)

	outname = [i.name for i in session.get_outputs()]
	outname
	inname = [i.name for i in session.get_inputs()]
	inname

	img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
	image = img.copy()
	image, ratio, dwdh = letterbox(image, auto=False)
	image = image.transpose((2, 0, 1))
	image = np.expand_dims(image, 0)
	image = np.ascontiguousarray(image)

	im = image.astype(np.float32)
	im /= 255
	im.shape
	inp = {inname[0]:im}

	outputs = session.run(outname, inp)[0]
	outputs
	ori_images = [img.copy()]

	number = 0

	thumb = None
	
	for i,(batch_id,x0,y0,x1,y1,cls_id,score) in enumerate(outputs):
		thumb = ori_images[int(batch_id)]
		cls_id = int(cls_id)
		if cls_id != 0 :
			# only keep person information
			continue
        
		# a person is found
		number += 1

		box = np.array([x0,y0,x1,y1])
		box -= np.array(dwdh*2)
		box /= ratio
		box = box.round().astype(np.int32).tolist()

		name = names[cls_id]
		color = colors[name]
		cv2.rectangle(thumb,box[:2],box[2:],color,2)
		# resize to 1/4
		thumb = cv2.resize(thumb, (0, 0), fx=0.25, fy=0.25)
		thumb = cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB)

	return number, thumb

for filename in glob.glob('../../Temp/*.mp4'):
	processVideo(filename, './Temp', './thumbnails/cache')


Starting to process 宋士杰, 11:23:33


FloatProgress(value=0.0, max=5820.8)

KeyboardInterrupt: 